In [20]:
import os
import random
import uuid
import numpy as np
from faker import Faker
from datetime import datetime
import time
import boto3
import json
from dotenv import load_dotenv

In [21]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [22]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [23]:
# Initialize Faker
fake = Faker('es-ES')

# Constants for random address generation in Madrid
LAT_MIN = 40.3121
LAT_MAX = 40.6521
LON_MIN = -3.8666
LON_MAX = -3.5673
max_vol_per_order = 500.0
max_weight_per_order = 100.0
status = "RECEIVED"

stream_name = 'OrderStreamForDispatching'  # Update with your Kinesis stream name

In [24]:
# Function to generate a random address in Madrid
def generate_random_address():
    street_name = fake.street_name()
    street_number = fake.building_number()
    lat = random.uniform(LAT_MIN, LAT_MAX)
    lon = random.uniform(LON_MIN, LON_MAX)
    address = f"{street_name}, {street_number}, Madrid, Spain"
    return {
        "address": address,
        "lat": lat,
        "lon": lon
    }

# Function to generate a random order
def generate_random_order(client_id):
    order_id = str(uuid.uuid4())
    volume = np.random.uniform(1, max_vol_per_order)
    weight = np.random.uniform(1, max_weight_per_order)
    n_objects = np.random.randint(1, 10)
    
    # Ensure the timestamp is generated in the correct format (Y-M-D H:M:S)
    order_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Ensure the total price is a random float and never 0
    total_price = np.random.uniform(10.0, 1000.0)
    
    order = {
        "order_id": order_id,
        "customer_id": f"cus-{uuid.uuid4()}",  # Simulating customer_id format
        "total_weight": weight,
        "total_volume": volume,
        "total_price": total_price,
        "order_timestamp": order_timestamp,  # Formatted timestamp
        "status": status,
        "lat": random.uniform(LAT_MIN, LAT_MAX),
        "lon": random.uniform(LON_MIN, LON_MAX)
    }
    print("Generated order:", order)  # Print the generated order for debugging
    return order

# Function to send the generated order to Kinesis
def send_order_to_kinesis(stream_name, order):
    order_data = json.dumps(order)
    
    response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=order_data.encode('utf-8'),
        PartitionKey=str(order['customer_id'])  # Partition based on customer_id
    )
    print(f"Order {order['order_id']} sent to Kinesis. Response: {response}")

# Function to generate orders at random intervals and send them to Kinesis
def order_stream_generator(stream_name):
    while True:
        client_id = random.randint(100000000000, 999999999999)
        order = generate_random_order(client_id)
        send_order_to_kinesis(stream_name, order)
        
        # Sleep for a random amount of time between 10 and 50 seconds (can be adjusted)
        sleep_time = random.randint(10, 30)  # Random sleep time in seconds
        print(f"Sleeping for {sleep_time / 60} minutes")
        time.sleep(sleep_time)


In [25]:
# Start the order stream generator
order_stream_generator(stream_name)

Generated order: {'order_id': '5a3646c0-fc0d-48cc-b20a-26da8ab5c07e', 'customer_id': 'cus-c85c717a-78fd-4c44-b151-14387d4b8d74', 'total_weight': 76.89711470276836, 'total_volume': 189.4248487210732, 'total_price': 934.4575800252746, 'order_timestamp': '2024-10-03 03:30:01', 'status': 'RECEIVED', 'lat': 40.34432315392608, 'lon': -3.756750482945305}
Order 5a3646c0-fc0d-48cc-b20a-26da8ab5c07e sent to Kinesis. Response: {'ShardId': 'shardId-000000000000', 'SequenceNumber': '49656295829544838090770636299804492719501347106072821762', 'ResponseMetadata': {'RequestId': 'd53aed8a-91c7-ec6b-8a4d-185091358b89', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd53aed8a-91c7-ec6b-8a4d-185091358b89', 'x-amz-id-2': 'qijPHxtBaKi/eEDrLmuR915zImgILzhL4cvFNRr/VztMomr1x+iz9hLu5EDw2IVOzZ1Ogn2bJjZHMtwcnf0EaUdRnlg5KeJM', 'date': 'Thu, 03 Oct 2024 01:30:02 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
Sleeping for 0.45

AccessDeniedException: An error occurred (AccessDeniedException) when calling the PutRecord operation: User: arn:aws:sts::277709186030:assumed-role/voclabs/user3002246=borja.gonzalez@alu.mbitschool.com is not authorized to perform: kinesis:PutRecord on resource: arn:aws:kinesis:us-east-1:277709186030:stream/OrderStreamForDispatching with an explicit deny in an identity-based policy